In [1]:
using Random
using Distributions
using FLoops
using Base.Threads

include("data.jl")
include("util.jl")

include("intervals/permutation.jl")
include("intervals/bootstrap.jl")
include("intervals/t.jl")

Threads.nthreads()

112

In [36]:
"""
DATA CONFIG
"""
alpha = 0.05

# data
B  = 100    # num. coverage probabilities per boxplot
S  = 1800   # num. samples per coverage probability
nx = 8      # size of group 1
ny = 8     # size of group 2
px, py = partition(nx, ny)
mc_size = try
    (binomial(nx+ny, nx) > 45_000) ? 10_000 : 0
catch
    0
end

dtype = Float32
seed = 123

"""
POPULATION SETTINGS
"""

Random.seed!(seed)
distrTypeX = Normal{dtype}
muX = dtype.(round.(rand(Uniform(0, 1), B), digits=3))
sdX = 2 .* muX
distrX = map(distrTypeX, muX, sdX)

distrTypeY = Normal{dtype}
#muY = dtype.(round.(rand(Uniform(0, 1), B), digits=4))
muY = 5 .+ muX
sdY = sdX
distrY = map(distrTypeY, muY, sdY)

@show distrX[1]
@show distrY[1]

deltas = @. mean(distrX) - mean(distrY);

distrX[1] = Normal{Float32}(μ=0.906f0, σ=1.812f0)
distrY[1] = Normal{Float32}(μ=5.906f0, σ=1.812f0)


In [42]:
Random.seed!(123)
xs = [rand(distrX[i], nx, S) for i in 1:B]
ys = [rand(distrY[i], nx, S) for i in 1:B]
@show size(ys)
ys[2]

size(ys) = (100,)


8×1800 Matrix{Float32}:
 5.86046  5.22366  7.10505  4.37502  …  4.25748  6.38194  6.41548  4.89421
 5.21914  5.70342  4.23227  6.58422     5.92762  4.71447  6.64843  6.27187
 5.91383  5.29252  5.06846  6.41311     5.37003  2.91424  6.31675  4.12682
 2.94526  6.66259  5.39992  4.50337     7.26296  5.39531  4.96408  6.39604
 4.79775  6.59313  4.15702  4.3229      6.47965  5.40516  4.18001  4.24418
 7.14234  4.75319  6.10638  5.42706  …  4.9405   7.44997  5.88326  4.22156
 5.72458  6.04158  6.206    6.99709     6.46472  5.23017  4.04794  5.77849
 5.19582  6.03821  4.70971  6.40662     5.92166  4.72116  5.77421  5.60228

In [43]:
# flatten into 3D matrix
X = reshape(hcat(xs...), 8, S, B)
Y = reshape(hcat(ys...), 8, S, B)
Y[:,:,2]

8×1800 Matrix{Float32}:
 5.86046  5.22366  7.10505  4.37502  …  4.25748  6.38194  6.41548  4.89421
 5.21914  5.70342  4.23227  6.58422     5.92762  4.71447  6.64843  6.27187
 5.91383  5.29252  5.06846  6.41311     5.37003  2.91424  6.31675  4.12682
 2.94526  6.66259  5.39992  4.50337     7.26296  5.39531  4.96408  6.39604
 4.79775  6.59313  4.15702  4.3229      6.47965  5.40516  4.18001  4.24418
 7.14234  4.75319  6.10638  5.42706  …  4.9405   7.44997  5.88326  4.22156
 5.72458  6.04158  6.206    6.99709     6.46472  5.23017  4.04794  5.77849
 5.19582  6.03821  4.70971  6.40662     5.92166  4.72116  5.77421  5.60228

In [65]:
B = 100
S = 1800

T = Threads.nthreads()
results = Array{Union{Nothing, Tuple}, 3}(undef, 6, B, S)
fill!(results, nothing)

#@time Threads.@threads for (i,j) in collect(Iterators.product(1:B, 1:S)) # 15.52 sec on (B,S) = (5, 1800)
@time @floop ThreadedEx(basesize=div(B*S, T)) for b in 1:B, s in 1:S
    @inbounds x = X[:,s,b]
    @inbounds y = Y[:,s,b]

    results[1, b, s] = permInterval(x, y, deltas[b], (px, py, mc_size), true, 0.05, twoSided, twoSided)    
    results[2, b, s] = permInterval(x, y, deltas[b], (px, py, mc_size), false, alpha, twoSided, twoSided)
    results[3, b, s] = permInterval(x, y, deltas[b], (px, py, mc_size), true, alpha/2, greater, smaller)
    results[4, b, s] = permInterval(x, y, deltas[b], (px, py, mc_size), false, alpha/2, greater, smaller)
    
    """
    lo, hi = bootstrap(x, y, 0.05, false, 10_000)
    coverages[5] += (lo <= delta <= hi)

    lo, hi = tconf(x, y, 0.05, false)
    coverages[6] += (lo <= delta <= hi)
    """
end

save_permutation_results(results, B, S)

2054.484222 seconds (1.12 G allocations: 12.301 TiB, 71.47% gc time, 0.00% compilation time)


In [51]:
averages = []
for row in eachrow(results[1,:,:])
    coverage = sum([j for (j, _) in row]) / S
    width = sum([j for (_, j) in row]) / S
    push!(averages, (coverage, width))
end
averages

5-element Vector{Any}:
 (0.9, 3.190407323161939)
 (0.8933333333333333, 1.5502733808778613)
 (0.8944444444444445, 2.586856261654738)
 (0.8977777777777778, 1.7984253204964482)
 (0.9033333333333333, 0.8845411716270898)

In [62]:
function save_permutation_results(results, B, S, dir="./")
    i = 1
    per_method = []
    for is_two_sided in [true, false]
        for pooled in [true, false]
            averages = []

            for row in eachrow(results[i,:,:])
                coverage = sum([j for (j, _) in row]) / S
                width = sum([j for (_, j) in row]) / S
                push!(averages, (coverage, width))
            end

            alpha_ = is_two_sided ? alpha : alpha / 2
            save(averages, distrX[1:B], distrY[1:B], alpha_, pooled, is_two_sided, parent_dir=dir)
            i += 1
        end
    end
end

save_permutation_results (generic function with 4 methods)

Alex Han

InvertingPermutationTests2022